In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset 
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
import os
import math
%run utils.ipynb


In [ ]:
hits, cells, particles, truth = load_data_single_event(1000)

In [ ]:
"""
    hits: tensor of samples x timesteps x features
    bounds: K:V of x_max, x_min, y_max, y_min, z_max, z_min
"""

"""
assuming input is samples x [x_val,y_val,z_val]
"""
def calc_norm(xyz, x_min, x_max, y_min, y_max, z_min, z_max):
    norm_x = ((xyz[0] - x_min)/(x_max - x_min))
    norm_y = ((xyz[1] - y_min)/(y_max - y_min))
    norm_z = ((xyz[2] - z_min)/(z_max - z_min))
    return [norm_x,norm_y,norm_z]

def norm_hits(hits, bounds):
    assert ('x_max' and 'x_min' and 'y_max' and 'y_min' and 'z_max' and 'z_min' in bounds) or ('r_max' and 'r_min' and 'phi_max' and 'phi_min' and 'z_max' and 'z_min' in bounds)
    return np.apply_along_axis(calc_norm, 1, hits, x_min = bounds['x_min'], x_max = bounds['x_max'], y_min = bounds['y_min'], y_max = bounds['y_max'], z_min = bounds['z_min'], z_max = bounds['z_max']) 


def calc_bounds():
    bounds={'x_max':0, 'x_min':0, 'y_max':0, 'y_min':0, 'z_max':0, 'z_min':0}
    for num in range(1000,1099):
        file_name = 'event00000' + str(num)
        hits, cells, particles, truth = load_event('data/train_sample/'+file_name)
        event_x_max, event_x_min, event_y_max, event_y_min, event_z_max, event_z_min = hits['x'].max(), hits['x'].min(), hits['y'].max(), hits['y'].min(), hits['z'].max(), hits['z'].min() 
        if bounds['x_max'] < event_x_max:
            bounds['x_max'] = event_x_max
        if bounds['x_min'] > event_x_min:
            bounds['x_min'] = event_x_min
        if bounds['y_max'] < event_y_max:
            bounds['y_max'] = event_y_max
        if bounds['y_min'] > event_y_min:
            bounds['y_min'] = event_y_min
        if bounds['z_max'] < event_z_max:
            bounds['z_max'] = event_z_max
        if bounds['z_min'] > event_y_min:
            bounds['z_min'] = event_y_min
    return bounds
        

"""
input: hits and truth dfs

output: df of particle_id as index and list of particles 
"""

def gen_tracks(truth_df):
    assert(isinstance(truth_df, pd.DataFrame))
    #print(truth_df)
    truth_df['dist'] = np.sqrt(truth_df['tx']**2+truth_df['ty']**2+truth_df['tz']**2)
    grouped = truth_df.groupby('particle_id')['hit_id','dist']
    a = grouped.apply(lambda x: x.sort_values(by=['dist'],ascending=True))
    final = a.groupby('particle_id')['hit_id'].apply(np.array)
    return final 

##there's still a problem with how to deal with hits that have a particle id of 0

def batch_iter(truth_df, batch_size):
    tracks = gen_tracks(truth_df).values
    np.random.shuffle(tracks) 
    remainder = len(tracks) % batch_size if len(tracks) % batch_size is not 0 else 0
    if remainder is not 0:
        modded_tracks = tracks[:-remainder]
    else:
        modded_tracks = tracks 
    assert(len(modded_tracks)%batch_size is 0)
    for batch in modded_tracks.reshape(-1,batch_size,1):
        yield batch
        
def get_data(max_seq_len, batch_size, feature_len, truth_df, hits_df):
    hits = hits_df
    max_seq_len = max_seq_len
    b_size = batch_size
    features = feature_len #xyz or phi r z
    all_data = list(batch_iter(truth_df,b_size))
    
    #print(all_data)
    for result in all_data:
        batch = []
        labels_tensor = []
        for track_list in result:
            for hit_id in track_list:
                hit_coord = []
                track = []
                for elem in hit_id:
                    x,y,z = hits.loc[hits['hit_id']== elem]['x'].item(), hits.loc[hits['hit_id']== elem]['y'].item(), hits.loc[hits['hit_id']== elem]['z'].item()
                    r,phi,z = cartesian_to_3d_polar(x,y,z)
                    hit_coord = [r,phi,z]
                    track.append(hit_coord)
                zeros_to_add = max_seq_len - len(track)
                if zeros_to_add > 0:
                    add_array = np.zeros((zeros_to_add,3))
                    np_data = np.array(track) 
                    padded_track_data  = np.append(np_data,add_array,axis=0)
                elif zeros_to_add < 0:
                    modded_track = track[:zeros_to_add]
                    padded_track_data = np.array(modded_track)
                else:
                    padded_track_data = np.array(track)
            row_label = padded_track_data[1:]
            padded_row_label = np.append(row_label, np.zeros((1,3)),axis=0)
            labels_tensor.append(padded_row_label)
            batch.append(padded_track_data)
        padded_batch_data = np.array(batch)
        padded_labels = np.array(labels_tensor)
        #print(padded_labels)
        yield padded_batch_data, padded_labels
        
def next_batch(max_seq_len, batch_size, feature_len):
    all_data = load_dataset('data/train_sample/', parts=['hits','truth'])
    for data in all_data:
        hit_df, truth_df = data[1], data[2]
        yield from get_data(max_seq_len, batch_size, feature_len, truth_df, hit_df)

n_steps = 7
n_input_features = 3
n_neurons = 200
n_outputs = 3

x = tf.placeholder(tf.float32, [None, n_steps, n_input_features] )
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])





#uses tanh for inner activations
lstm = tf.contrib.rnn.LSTMCell(num_units = n_neurons, use_peepholes = True)

lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = n_outputs)
rnn_outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype = tf.float32)

init = tf.global_variables_initializer()

sess = tf.Session()
saver = tf.train.import_meta_graph('./checkpoints-61300.meta')
saver.restore (sess, tf.train.latest_checkpoint ('./'))
graph = tf.get_default_graph()

n_iterations = 100000

#saver = tf.train.Saver()
#tf.reset_default_graph()
        
with tf.Session() as sess:
   
    init.run()
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    for iteration in range(n_iterations):
        x_data, y_data = next(next_batch(n_steps, (n_iterations*100), n_input_features))
        _, loss_batch, summary = sess.run([train_op, batch_loss, summary_op], feed_dict={x:x_data, y:y_data})
        writer.add_summary(summary, global_step=iteration)
        
        if iteration % 100 ==0:
            mse = batch_loss.eval(feed_dict={X:x_batch, y:y_batch})
            print(iteration, "\tMSE:", mse)
            saver.save(sess, './checkpoints', iteration)
    #saver.save(sess,"./lstm_weights1.ckpt")

ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints'))

saver.restore(sess, ckpt.model_checkpoint_path)


In [2]:
init = tf.global_variables_initializer()

n_steps = 7
n_input_features = 5
n_neurons = 200
n_outputs = 3
restoreGraph = tf.train.import_meta_graph('checkpoints-0.meta')
with tf.Session() as infSess:
    print('hello?')
    restoreGraph.restore(infSess, 'checkpoints-0')

    graph = tf.get_default_graph()
    #print('here?')
    #inputs = graph.get_tensor_by_name("input:0")
    inputs = tf.placeholder(tf.float32, [None, 7,5 ])
    
    #label_op = graph.get_tensor_by_name("label")
    #print('how about now?')
    opt_to_restore = graph.get_tensor_by_name("print_pred:0")
    for i in range(100):
        #print('helloooooooooo?')
        x_data, y_data, _, __ = next(next_batch(n_steps, 1, n_input_features))
        #print(x_data)
        #print(y_data)
        show_pred = infSess.run(opt_to_restore, feed_dict={inputs:x_data})
        print('input is:')
        print(x_data)
        print('model pred is:')
        print(show_pred)
        print('true track is:')
        print(y_data)
        
        
        

        

                                                  

hello?
INFO:tensorflow:Restoring parameters from checkpoints-0


InvalidArgumentError: You must feed a value for placeholder tensor 'input' with dtype float and shape [?,7,3]
	 [[Node: input = Placeholder[dtype=DT_FLOAT, shape=[?,7,3], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: rnn/transpose_1/_57 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_151_rnn/transpose_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input', defined at:
  File "/home/dl-one/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/dl-one/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dl-one/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/dl-o

In [ ]:
%pwd

In [ ]:
%ls checkpoints-*